In [284]:
# Libraries
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import pandas as pd
from time import sleep

# Selenium Webdriver
options = Options()
options.headless = True
assert options.headless
driver = webdriver.Firefox(options=options)

In [303]:
# Returns number of following users
SC_USERNAME = 'glut_it'
driver.get('https://soundcloud.com/' + SC_USERNAME)

following = driver.find_element_by_xpath("//*[starts-with(@class,'infoStats__stat')]//*[text()='Following']//following::div")

# Convert followers from string to integer
if following is not None:
    following = int(re.sub("\D", "", following.text))
    print("Following: ", following)
else:
    raise Exception("No Followers Found!")

Following:  2003


In [292]:
# Scroll to the end for infinite dynamic scrolling
SCROLL_PAUSE_TIME = 0.5
FACTOR = 0.9
driver.get('https://soundcloud.com/' + SC_USERNAME + '/following')

from IPython.display import display, clear_output

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")
# Loop if new scroll height is not equal to current scroll height
while True:
    # Display count of loaded profiles
    clear_output(wait = True)
    print("Retrieved: ", len(driver.find_elements_by_xpath("//div[@class='userBadgeListItem']")))
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load page
    sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if (new_height == last_height) and (len(driver.find_elements_by_xpath("//div[@class='userBadgeListItem']")) > (FACTOR * following)):
        break
    last_height = new_height

Retrieved:  151


In [293]:
# Returns webdriver elements for required info
profile = driver.find_elements_by_xpath("//div[@class='userNetwork']//a[starts-with(@class,'userBadgeListItem__heading')]")
image = driver.find_elements_by_xpath("//div[@class='userNetwork']//span[starts-with(@class,'sc-artwork')]")
print(following, len(profile),len(image))

155 151 151


In [301]:
# Returns dataframe from webdriver with text info
temp1, temp2, temp3 = [], [], []

# Returns html for profile thumbnail
for x in range(len(image)):
    # Error check if there is thumbnail
    if 'url' in image[x].get_attribute('style'):
        temp1.append((image[x].get_attribute('style')).split('"')[1])
    else:
        temp1.append(None)
# Returns username and profile html    
for x in range(len(profile)):
    temp2.append(profile[x].text)
    temp3.append(profile[x].get_attribute('href'))
# Convert list to DataFrame    
df = (pd.DataFrame({'image-url':temp1, 'username':temp2, 'profile':temp3}))
# Export to CSV
df.to_csv('automation '+SC_USERNAME+' following.csv', index=False, header=False)#, mode='a')

In [281]:
from IPython.display import Image
Image("https://i1.sndcdn.com/avatars-000395181261-hfn1rr-t200x200.jpg")

In [282]:
# Close the webdriver session
driver.close()